# ========================================================================================
# 2m_notebook_5
# ========================================================================================

In [1]:
import pandas as pd
import os
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

В данной задаче ссылка для считывания данных лежит в переменной path_to_file.  
Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [ ]:
retail = pd.read_csv(r'path_to_file', encoding='ISO-8859-1')

In [ ]:
retail_columns = retail.columns

Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [ ]:
duplicates = retail[retail.duplicated(keep=False)]

In [ ]:
len(retail) - len(retail.drop_duplicates())

In [ ]:
retail = retail.drop_duplicates()

Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего транзакций отменили пользователи? Считайте, что каждая строка - это отдельная транзакция.

In [ ]:
len(retail[retail['InvoiceNo'].str.startswith('C')])

Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк

In [ ]:
retail = retail[retail.Quantity > 0]

In [ ]:
retail.shape

Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

In [ ]:
retail_germany = retail[retail.Country == 'Germany']\
    .groupby('CustomerID')\
    .agg({'InvoiceNo': 'nunique'})

In [ ]:
N = retail_germany.quantile(q=0.8)

In [ ]:
germany_top = retail_germany[retail_germany.InvoiceNo > N.InvoiceNo].index.tolist()

In [ ]:
invoice_dist = retail[retail.Country == 'Germany']\
    .groupby('CustomerID', as_index=False)\
    .agg(total_invoices = ('InvoiceNo', 'nunique'))

In [ ]:
N = invoice_dist.total_invoices.quantile(q=0.8)

In [ ]:
germany_top = invoice_dist.loc[invoice_dist.total_invoices > N]['CustomerID']

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [ ]:
top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]

Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?   
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [ ]:
top_retail_germany.groupby('StockCode')\
    .size()\
    .value_counts()\
    .sort_values(ascending=False)

Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [ ]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [ ]:
top_revenue = retail.groupby('InvoiceNO')\
    .agg(TotalRevenue = ('Revenue', 'sum'))\
    .sort_values()\
    .head()\
    .index